In [1]:
import numpy as np
import time
from enum import IntEnum
from pynq import  Overlay, DefaultHierarchy, allocate, DefaultIP, MMIO
import logging
from contextlib import contextmanager


In [3]:

# -------------------- Configure the logger --------------------
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
time_logger = logging.getLogger('time')
time_logger.setLevel(logging.INFO)

# ---------------------- Timer ----------------------
@contextmanager
def time_measure(name):
    start = time.perf_counter()
    yield
    elapsed = (time.perf_counter() - start) * 1000  # 毫秒
    time_logger.info(f"{name.ljust(30)}: {elapsed:.3f} ms")



# ---------------------- CryptoDriver ----------------------
# Define register addresses and operation codes
class CryptoRegisters(IntEnum):
    CONTROL = 0x0000
    RAM_SELECT = 0x0010
    RAM_SELECT1 = 0x0018
    OP_CODE = 0x0020

class OpCode(IntEnum):
    ADD_MOD                     = 0
    SUB_MOD                     = 1
    MUL_MOD                     = 2
    NTT                         = 3
    INTT                        = 4
    POLY_WRITE                  = 5
    POLY_READ                   = 6
    TWIDDLE_WRITE               = 7
    POLY_MOD_PLAINTEXTMODULUS   = 8

In [4]:
overlay = Overlay("/home/xilinx/pynq/overlays/Crypto/Crypto.bit")
dma = overlay.axi_dma_0
crypto = overlay.Crypto_0

In [12]:
# 创建分配缓冲区
POLY_DEGREE = 4096
DELTA = 4
delta = allocate(shape=(POLY_DEGREE * 3,), dtype=np.int32)
delta[:] = [DELTA] * POLY_DEGREE * 3
print(delta)

# 写入数据到Crypto IP
print("Writing data to Crypto IP")
with time_measure("Write data to Crypto IP"):
    crypto.write(CryptoRegisters.RAM_SELECT, 0)
    crypto.write(CryptoRegisters.OP_CODE, int(OpCode.POLY_WRITE))
    crypto.write(CryptoRegisters.CONTROL, 1)
    dma.sendchannel.transfer(delta)
    print("Waiting for DMA to finish")
    dma.sendchannel.wait()
    print("DMA send completed")



INFO:time:Write data to Crypto IP       : 1.418 ms


[4 4 4 ... 4 4 4]
Writing data to Crypto IP
Waiting for DMA to finish
DMA send completed


In [ ]:
# 读取数据
print("Reading data from Crypto IP")
output = [0] * POLY_DEGREE * 3
with time_measure("Read data from Crypto IP"): 
    crypto.write(CryptoRegisters.RAM_SELECT, 0)
    crypto.write(CryptoRegisters.OP_CODE, int(OpCode.POLY_READ))

    for i in range(POLY_DEGREE * 3):
        output[i] = crypto.read(0x10000 + i * 4)
    crypto.write(CryptoRegisters.CONTROL, 1)
print(output)
    

Reading data from Crypto IP
[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,